In [1]:
!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb libav-tools xorg-dev\
 python-opengl libboost-all-dev libsdl2-dev swig 1> /dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version.
libjpeg-dev is already the newest version.
libjpeg-dev set to manually installed.
python-dev is already the newest version.
zlib1g-dev is already the newest version.
zlib1g-dev set to manually installed.
The following extra packages will be installed:
  freeglut3 icu-devtools libasound2 libasound2-data libasound2-dev libasyncns0
  libavahi-client-dev libavahi-client3 libavahi-common-data
  libavahi-common-dev libavahi-common3 libavcodec54 libavdevice53 libavfilter3
  libavformat54 libavresample1 libavutil52 libboost-atomic-dev
  libboost-atomic1.54-dev libboost-atomic1.54.0 libboost-chrono-dev
  libboost-chrono1.54-dev libboost-chrono1.54.0 libboost-context-dev
  libboost-context1.54-dev libboost-context1.54.0 libboost-coroutine-dev
  libboost-coroutine1.54-dev libboost-date-time-dev libboost-date-time1.54-dev
  libboost-date-time1.54.0 libboost-dev libbo

In [2]:
!pip install gym 1> /dev/null

# environments 
!pip install gym[atari] 1> /dev/null

Downloading/unpacking gym
  Running setup.py (path:/tmp/pip_build_root/gym/setup.py) egg_info for package gym
    
Downloading/unpacking pyglet>=1.2.0 (from gym)
  Running setup.py install for gym
      File "/usr/local/lib/python2.7/dist-packages/gym/utils/reraise_impl_py3.py", line 4
        raise e.with_traceback(traceback) from None
                                             ^
    SyntaxError: invalid syntax
    
    Sorry: IndentationError: unexpected indent (meta_doom.py, line 2)
    
  Could not find .egg-info directory in install record for gym
Cleaning up...
  Installing extra requirements: 'atari'
Downloading/unpacking atari_py>=0.0.17 (from gym[atari])
  Running setup.py (path:/tmp/pip_build_root/atari-py/setup.py) egg_info for package atari-py
    
  Running setup.py install for atari-py
    make: Entering directory `/tmp/pip_build_root/atari-py/atari_py/ale_interface'
    mkdir -p build && cd build && cmake .. && make -j4
    -- The C compiler identification is GNU 4.8.4

In [3]:
!pip install gym[box2d] #1> /dev/null

  Installing extra requirements: 'box2d'
Downloading/unpacking box2d-py (from gym[box2d])
  Running setup.py (path:/tmp/pip_build_root/box2d-py/setup.py) egg_info for package box2d-py
    Using setuptools (version 24.0.2).
    
  Running setup.py install for box2d-py
    Using setuptools (version 24.0.2).
    building 'Box2D._Box2D' extension
    swigging Box2D/Box2D.i to Box2D/Box2D_wrap.cpp
    swig -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir library/Box2D -keyword -w511 -D_SWIG_KWARGS -o Box2D/Box2D_wrap.cpp Box2D/Box2D.i
    Box2D/Common/b2Math.h:67: Warning 302: Identifier 'b2Vec2' redefined by %extend (ignored),
    Box2D/Box2D_math.i:47: Warning 302: %extend definition of 'b2Vec2'.
    Box2D/Common/b2Math.h:158: Warning 302: Identifier 'b2Vec3' redefined by %extend (ignored),
    Box2D/Box2D_math.i:168: Warning 302: %extend definition of 'b2Vec3'.
    Box2D/Common/b2Math.h:197: Warning 302: Identifier 'b2Mat22' redefined by %extend 

In [4]:
!pip install gym[classic_control] #1> /dev/null

  Installing extra requirements: 'classic_control'
Cleaning up...


LunarLander-v2  CarRacing-v0

In [5]:
import numpy as np
import cPickle as pickle
import gym

In [6]:
env = gym.make('FrozenLake-v0')
env.reset()

[2016-07-11 02:22:46,763] Making new env: FrozenLake-v0


0

In [8]:
out = env.render(mode="human") #human / rgb_array / ansi

SFFF
FHFH
FFFH
HFFG



In [10]:
env.reset()
for _ in range(10):
    out = env.render(mode="human")
#     print(out.readlines())
    env.step(env.action_space.sample()) # take a random action

SFFF
FHFH
FFFH
HFFG

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)


Using: [this gist](https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5), referenced in [this Karpathy's post](http://karpathy.github.io/2016/05/31/rl/)

In [4]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import cPickle as pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    model['W2'] = np.random.randn(H) / np.sqrt(H)
    
grad_buffer = { k : np.zeros_like(v) for k,v in model.iteritems() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.iteritems() } # rmsprop memory

def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
    if render: env.render()

    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

        # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

        # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.iteritems():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

        # boring book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print 'resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward)
        if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None

    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print ('ep %d: game finished, reward: %f' % (episode_number, reward)) + ('' if reward == -1 else ' !!!!!!!!')


[2016-06-20 02:34:21,315] Making new env: Pong-v0


ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, reward: -1.000000
ep 1: game finished, rewar

KeyboardInterrupt: 